In [1]:
import streamlit as st
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import time
from azure.storage.blob import BlobServiceClient
import logging


logger = logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

connection_string = st.secrets["database_connection"]["connection_string"]
container_name = st.secrets["database_connection"]["container_name"]
masterSearchFileName = st.secrets["database_connection"]["masterSearchFileName"]
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container=container_name)

downlodData = container_client.download_blob(masterSearchFileName).readall()
keydict = pd.read_json(BytesIO(downlodData), dtype={"Search":str, "Time":np.float32, "Key":str})

search_for_orig = "trimmer"
# search_for_orig = st.text_input(label='Search for:', value='')
search_for_orig = search_for_orig.strip().lower()

num_prod_search = 160
displayBox = None
refresh = False

In [2]:
platform = "Amazon"
search_for = search_for_orig.replace(' ', '+')
base_link = ("https://www.amazon.in/s?k=" + search_for + 
            "&rh=p_72%3A1318476031")

df_list = []
progress = 0.0
page1 = 0
num_prod1 = 0
restime = time.time()
formatime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(restime))

In [6]:
link = base_link+"&page="+str(0)
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
           "DNT":"1","Connection":"close", 
           "Upgrade-Insecure-Requests":"1"} 
html_text = "Service Unavailable"
req_num = 0
while ("Service Unavailable" in html_text[:50]) & (req_num < 10):
    print(req_num)
    html_text = requests.get(link, headers=headers).text
    time.sleep(0.5)
    req_num = req_num+1
soup = BeautifulSoup(html_text, "html.parser")

0
1
2
3
4
5
6
7
8
9


In [19]:
html_text

'<html>\n<head>\n<title>503 - Service Unavailable Error</title>\n</head>\n<body bgcolor="#FFFFFF" text="#000000">\n\n<!--\n        To discuss automated access to Amazon data please contact api-services-support@amazon.com.\n        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.in/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.in/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.\n-->\n\n<center>\n<a href="https://www.amazon.in/ref=cs_503_logo/">\n<img src="https://images-eu.ssl-images-amazon.com/images/G/31/x-locale/communities/people/logo.gif" width=200 height=45 alt="Amazon.in" border=0></a>\n<p align=center>\n<font face="Verdana,Arial,Helvetica">\n<font size="+2" color="#CC6600"><b>Oops!</b></font><br>\n<b>It\'s rush hour and traffic is piling up on that page. Please try again in a short while.<br>If you were trying to place an order, it will not have been processe

In [3]:
base_link = ("https://www.amazon.in/s?k=" + "trimmer")
link = base_link+"&page="+ str(2) +"&qid=1712573300&ref=sr_pg_2"
link

'https://www.amazon.in/s?k=trimmer&page=2&qid=1712573300&ref=sr_pg_2'

In [12]:
link="https://www.amazon.in/s?k=trimmer&page=2&qid=1712573300&ref=sr_pg_2"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
           "Accept": "*/*", 
           "Accept-Encoding": "gzip, deflate, br, zstd", 
           "Accept-Language": "en,en-IN;q=0.9,en-US;q=0.8,hi;q=0.7", 
           "Referer": "https://www.amazon.in/",
           "Content-Type":"text/plain;charset=UTF-8",
           "Sec-Ch-Ua-Mobile":"?0",
           "Sec-Ch-Ua-Platform":"Windows",
           "Sec-Ch-Ua":'"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"'
           }

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
    "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"
}
html_text = requests.get(link, headers=headers).text

In [21]:
import pyarrow.lib as _lib

ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [13]:
html_text

'<!doctype html><html lang="en-in" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com" crossorigin>\n<link rel="preconnect" href="https://images-eu.ssl-images-amazon.com" crossorigin>\n<link rel="dns-prefetch" href="https://m.media-amazon.com" crossorigin>\n<link rel="preconnect" href="https://m.media-amazon.com" crossorigin>\n<link rel="dns-prefetch" href="https://completion.amazon.com" crossorigin>\n<link rel="preconnect" href="https://completion.amazon.com" crossorigin>\n<!-- sp:end-feature:cs-optimization -->\n<!-- s

In [14]:
soup = BeautifulSoup(html_text, "html.parser")

In [16]:
prods = soup.find_all("div", class_="a-section a-spacing-base")

In [17]:
prods

[<div class="a-section a-spacing-base"><div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-vbok7i09ua2q62ek5q2l21tt78"><span class="rush-component" data-component-type="s-product-image" data-render-id="r2y6wav4tpjmno254lr8evdiqo4" data-version-id="vbok7i09ua2q62ek5q2l21tt78"><a class="a-link-normal s-no-outline" href="/Philips-Rechargeable-Technology-MG5930-65/dp/B0CXDTQCP3/ref=sr_1_49?dib=eyJ2IjoiMSJ9.Rm0hMVZQtLTo-OHYADuujjaJy9Vvx9Rz5oniUsi_XmQ7qGmLdoD8NR9eaKny-2DeS3InDxsgbwU4AKG_7vmavtIZHCD9lmXqFQgpnyQuSXi--M04ua26wfn8Lz1uVej5m0c_9Uit6D_3SbCl-CDJHbt7xBZ0j4OhS03_gVfznlqklc2hsfo3V7G77ytwONRV57hyMxLaIZ2TkiNAW2IP8VvR4gXRZgiWxfWeH6sKzpkZ6YtL7gff9vwuc93RR1cOHLBP2oCp7tQk8aWvGEhUBtiO2Jq-69rdd5HeNZBZhH4.T16L-MKUrpS1giYZyQIwjMWtpJnQXDgNkwaPg6f43f0&amp;dib_tag=se&amp;keywords=trimmer&amp;qid=1712576794&amp;sr=8-49" target="_blank"><div class="a-sec

In [18]:
df_list

[]

In [19]:
for prod in prods:
    desc_box1 = prod.find("h5", class_="s-line-clamp-1")
    desc_box2 = prod.find("span", class_="a-size-base-plus a-color-base a-text-normal")
    desc_box3 = prod.find("span", class_="a-size-medium a-color-base a-text-normal")
    descrs = ""
    if (desc_box1 == None) and (desc_box2 == None) and (desc_box3 == None):
        continue
    if desc_box1 != None:
        descrs = desc_box1.text + " | "
    if desc_box2 != None:
        descrs = descrs + desc_box2.text
    if desc_box3 != None:
        descrs = descrs + desc_box3.text
    prodLink = "https://www.amazon.in" + (prod.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href'].split("/ref=")[0])
    pricebox = prod.find("span", class_="a-price-whole")
    if pricebox == None:
        continue
    price = int(round(float(pricebox.text.replace(',', '')), 0))
    ratebox = prod.find("span", class_="a-icon-alt")
    if ratebox == None:
        continue
    rating = float(ratebox.text.split()[0])
    raters = int(prod.find("span", class_="a-size-base s-underline-text").text.replace(',', '').replace('(', '').replace(')', ''))
    if raters < 30:
        continue
    img_url = prod.find("img", class_="s-image")['src']
    df_list.append([platform, descrs, prodLink, price, rating, raters, np.nan, img_url])
    num_prod1 = num_prod1 + 1
    print(f"Amazon : searched for {num_prod1} products in {page1} pages")
    progress = progress + (100/(num_prod_search))
    continue

Amazon : searched for 1 products in 0 pages
Amazon : searched for 2 products in 0 pages
Amazon : searched for 3 products in 0 pages
Amazon : searched for 4 products in 0 pages
Amazon : searched for 5 products in 0 pages
Amazon : searched for 6 products in 0 pages
Amazon : searched for 7 products in 0 pages
Amazon : searched for 8 products in 0 pages
Amazon : searched for 9 products in 0 pages
Amazon : searched for 10 products in 0 pages
Amazon : searched for 11 products in 0 pages
Amazon : searched for 12 products in 0 pages
Amazon : searched for 13 products in 0 pages
Amazon : searched for 14 products in 0 pages
Amazon : searched for 15 products in 0 pages
Amazon : searched for 16 products in 0 pages
Amazon : searched for 17 products in 0 pages
Amazon : searched for 18 products in 0 pages
Amazon : searched for 19 products in 0 pages
Amazon : searched for 20 products in 0 pages
Amazon : searched for 21 products in 0 pages
Amazon : searched for 22 products in 0 pages
Amazon : searched f

In [20]:
df_list

[['Amazon',
  'VGR V-055 Professional Cordless Rechargeable Beard Trimmer Hair Clippers for Men with Guide Combs Brush, Black',
  'https://www.amazon.in/VGR-V-055-Professional-Cordless-Rechargeable/dp/B08JNP488Z',
  674,
  4.1,
  5083,
  nan,
  'https://m.media-amazon.com/images/I/61YC597sanL._AC_UL320_.jpg'],
 ['Amazon',
  'Havells Electric Beard Trimmer | IPX7 Fully Washable | 120 minutes Runtime | Upto 13mm Length Settings | Cord & Cordless Use | 2 Years Guarantee | Brown | BT5115',
  'https://www.amazon.in/Havells-BT5115-Electric-Washable-Guarantee/dp/B0BZDFV2H1',
  1139,
  3.5,
  178,
  nan,
  'https://m.media-amazon.com/images/I/61u8mNn-G-L._AC_UL320_.jpg'],
 ['Amazon',
  'Bombay Shaving Company Premium Multi Grooming Trimmer for Men | Body and Beard Trimmer for Man | Multi Styling Rotary Comb, IPX6 Waterproof, 90 Min Run Time, 2 Hrs Charge Time | 1 PC',
  'https://www.amazon.in/Bombay-Shaving-Company-Grooming-Waterproof/dp/B0CKLP826Z',
  1999,
  4.0,
  139,
  nan,
  'https://m.m

In [ ]:
platform = "Amazon"
search_for = search_for_orig.replace(' ', '+')
base_link = ("https://www.amazon.in/s?k=" + search_for + 
            "&rh=p_72%3A1318476031")

df_list = []
progress = 0.0
page1 = 0
num_prod1 = 0
restime = time.time()
formatime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(restime))

while progress < 100:
    page1 = page1 + 1
    if page1 == 10:
        break
    link = base_link+"&page="+str(page1)
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 
    html_text = "Service Unavailable"
    req_num = 0
    while ("Service Unavailable" in html_text[:50]) & (req_num < 10):
        print(req_num)
        html_text = requests.get(link, headers=headers).text
        time.sleep(0.5)
        req_num = req_num+1
    soup = BeautifulSoup(html_text, "html.parser")
    # print("\n\nPage :", page)
    # print("link:")
    # print(link, "\n\n")

    prods = soup.find_all("div", class_="a-section a-spacing-base")
    if len(prods) == 0:
        prods = soup.find_all("div", class_="a-section a-spacing-base a-text-center")
    if len(prods) == 0:
        prods = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ["a-section"])
    
    for prod in prods:
        desc_box1 = prod.find("h5", class_="s-line-clamp-1")
        desc_box2 = prod.find("span", class_="a-size-base-plus a-color-base a-text-normal")
        desc_box3 = prod.find("span", class_="a-size-medium a-color-base a-text-normal")
        descrs = ""
        if (desc_box1 == None) and (desc_box2 == None) and (desc_box3 == None):
            continue
        if desc_box1 != None:
            descrs = desc_box1.text + " | "
        if desc_box2 != None:
            descrs = descrs + desc_box2.text
        if desc_box3 != None:
            descrs = descrs + desc_box3.text
        prodLink = "https://www.amazon.in" + (prod.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href'].split("/ref=")[0])
        pricebox = prod.find("span", class_="a-price-whole")
        if pricebox == None:
            continue
        price = int(round(float(pricebox.text.replace(',', '')), 0))
        ratebox = prod.find("span", class_="a-icon-alt")
        if ratebox == None:
            continue
        rating = float(ratebox.text.split()[0])
        raters = int(prod.find("span", class_="a-size-base s-underline-text").text.replace(',', '').replace('(', '').replace(')', ''))
        if raters < 30:
            continue
        img_url = prod.find("img", class_="s-image")['src']
        df_list.append([platform, descrs, prodLink, price, rating, raters, np.nan, img_url])
        num_prod1 = num_prod1 + 1
        print(f"Amazon : searched for {num_prod1} products in {page1} pages")
        progress = progress + (100/(num_prod_search))
        continue

In [2]:
link = ("https://www.amazon.in/s?k=" + "non+stick+cooking+set" + 
                    "&rh=p_72%3A1318476031") + "&page=1"

In [13]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 
html_text = requests.get(link, headers=headers).text
soup = BeautifulSoup(html_text, "html.parser")

0
1


In [46]:

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 
html_text = requests.get(link, headers=headers).text
soup = BeautifulSoup(html_text, "html.parser")

prods = soup.find_all("div", class_="a-section a-spacing-base")
if len(prods) == 0:
    prods = soup.find_all("div", class_="a-section a-spacing-base a-text-center")
if len(prods) == 0:
    prods = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ["a-section"])

prod = prods[0]

desc_box1 = prod.find("h5", class_="s-line-clamp-1")
desc_box2 = prod.find("span", class_="a-size-base-plus a-color-base a-text-normal")
desc_box3 = prod.find("span", class_="a-size-medium a-color-base a-text-normal")
descrs = ""
if (desc_box1 == None) and (desc_box2 == None) and (desc_box3 == None):
    print("ended at 1")
if desc_box1 != None:
    descrs = desc_box1.text + " | "
if desc_box2 != None:
    descrs = descrs + desc_box2.text
if desc_box3 != None:
    descrs = descrs + desc_box3.text
prodLink = "https://www.amazon.in" + (prod.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href'].split("/ref=")[0])
pricebox = prod.find("span", class_="a-price-whole")
if pricebox == None:
    print("ended at 2")
price = int(round(float(pricebox.text.replace(',', '')), 0))
ratebox = prod.find("span", class_="a-icon-alt")
if ratebox == None:
    print("ended at 3")
rating = float(ratebox.text.split()[0])
raters = int(prod.find("span", class_="a-size-base s-underline-text").text.replace(',', '').replace('(', '').replace(')', ''))
if raters < 30:
    print("ended at 4")
img_url = prod.find("img", class_="s-image")['src']

[descrs, prodLink, price, rating, raters, np.nan, img_url]

['MOKOBARA Backpack 15.5" Inch Laptop Backpack For Women (16 litres ) Improved (2022 - Model)',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyMTgwMTkyOTYyNzgzOTAzOjE2ODE4Mzk3MTk6c3BfYXRmOjIwMTE3Nzk1ODc0NDk4OjowOjo&url=%2FMokobara-Backpack-Leather-Resistant-College%2Fdp%2FB09WVGDV1V%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dleather%2Bbag%26qid%3D1681839719%26refinements%3Dp_72%253A1318476031%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DA2ZJIE4OXD710M',
 4499,
 4.3,
 158,
 nan,
 'https://m.media-amazon.com/images/I/714g-hjkkqL._AC_UY218_.jpg']

In [3]:
link

'https://www.amazon.in/s?k=leather+bag&rh=p_72%3A1318476031&page=1'

In [35]:
prods = soup.find_all("div", class_="a-section", partial=False)

In [38]:
prods = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ["a-section"])

In [37]:
prods[12]

<div class="a-section a-spacing-none _bXVsd_block_1vI8- _bXVsd_row_3CEm0 _bXVsd_itemsBaseline_3lQs7"><span class="a-price _bXVsd_price_23Ix_" data-a-color="base" data-a-size="medium_plus"><span class="a-offscreen">₹2,629.00</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">2,629<span class="a-price-decimal">.</span></span><span class="a-price-fraction">00</span></span></span> <i aria-label="Eligible for Prime." class="a-icon a-icon-prime a-icon-small _bXVsd_prime_1v0bf" role="img"></i></div>

In [39]:
len(prods)

23

In [10]:
prod = prods[24]

In [12]:
desc_box1 = prod.find("h5", class_="s-line-clamp-1")
desc_box2 = prod.find("span", class_="a-size-base-plus a-color-base a-text-normal")
descrs = ""
if (desc_box1 == None) and (desc_box2 == None):
    print("ended at 1")
if desc_box1 != None:
    descrs = desc_box1.text + " | "
if desc_box2 != None:
    descrs = descrs + desc_box2.text
prodLink = "https://www.amazon.in" + (prod.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href'].split("/ref=")[0])
pricebox = prod.find("span", class_="a-price-whole")
if pricebox == None:
    print("ended at 2")
price = int(round(float(pricebox.text.replace(',', '')), 0))
ratebox = prod.find("span", class_="a-icon-alt")
if ratebox == None:
    print("ended at 3")
rating = float(ratebox.text.split()[0])
raters = int(prod.find("span", class_="a-size-base s-underline-text").text.replace(',', '').replace('(', '').replace(')', ''))
if raters < 30:
    print("ended at 4")
img_url = prod.find("img", class_="s-image")['src']

ended at 1


TypeError: 'NoneType' object is not subscriptable

In [50]:
[descrs, prodLink, price, rating, raters, np.nan, img_url]

["Van Heusen | Men's Regular Fit Polo",
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToyMjQ2NDQ5MTg5NDQyNjQyOjE2ODExMTMwNTE6c3BfYXRmOjIwMTQzMjA0NjA4NDk4OjowOjo&url=%2FVan-Heusen-Sport-Regular-VSKPWRGFD34687_Rouge%2Fdp%2FB07HJHNMSN%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dtshirt%2Bfor%2Bmen%26qid%3D1681113051%26refinements%3Dp_72%253A1318476031%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 550,
 4.0,
 29142,
 nan,
 'https://m.media-amazon.com/images/I/811WFiZ+QQL._AC_UL400_.jpg']

In [40]:
prod = prods[2]

In [42]:
descrs = prod.find("span", class_="a-size-medium a-color-base a-text-normal").text
prodLink = "https://www.amazon.in" + prod.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")['href'].split("/ref=")[0]
pricebox = prod.find("span", class_="a-price-whole")
if pricebox == None:
    print("ended at 1")
price = int(round(float(pricebox.text.replace(',', '')), 0))
ratebox = prod.find("span", class_="a-icon-alt")
if ratebox == None:
    print("ended at 2")
rating = float(ratebox.text.split()[0])
raters = int(prod.find("span", class_="a-size-base s-underline-text").text.replace(',', '').replace('(', '').replace(')', ''))
if raters < 30:
    print("ended at 3")
img_url = prod.find("img", class_="s-image")['src']

In [20]:
prod.find("a", class_="a-section aok-relative s-image-fixed-height")

In [25]:
prod.prettify()

'<div class="a-section a-spacing-small a-spacing-top-small">\n <div class="a-section a-spacing-none puis-padding-right-small s-title-instructions-style">\n  <div class="a-row a-spacing-micro">\n   <span class="a-declarative" data-a-popover=\'{"closeButton":"true","dataStrategy":"preload","name":"sp-info-popover-B09WVGDV1V","position":"triggerVertical"}\' data-action="a-popover" data-csa-c-func-deps="aui-da-a-popover" data-csa-c-type="widget">\n    <a aria-label="View Sponsored information or leave ad feedback" class="puis-label-popover puis-sponsored-label-text" href="javascript:void(0)" role="button" style="text-decoration: none;">\n     <span class="puis-label-popover-default">\n      <span class="a-color-secondary">\n       Sponsored\n      </span>\n     </span>\n     <span class="puis-label-popover-hover">\n      <span class="a-color-base">\n       Sponsored\n      </span>\n     </span>\n     <span class="aok-inline-block puis-sponsored-label-info-icon">\n     </span>\n    </a>\n  

In [26]:
".jpg" in prod

False

In [43]:
[descrs, prodLink, price, rating, raters, np.nan, img_url]

['Fur Jaden Brown Leatherette Anti Theft Casual Laptop Backpack with 15.6 Inch Laptop Pocket',
 'https://www.amazon.in/Fur-Jaden-Backpack-Trekking-BM38_Brown/dp/B07LBMV4NR',
 1729,
 4.2,
 1530,
 nan,
 'https://m.media-amazon.com/images/I/714Rv25H9PL._AC_UY218_.jpg']